<a href="https://colab.research.google.com/github/Olena299/Python-Projects/blob/main/Portfolio_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0


In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd


In [ ]:
auth.authenticate_user()


In [ ]:
client = bigquery.Client(project="data-analytics-mate")


In [ ]:
query = """
WITH
  session_info AS (
  SELECT
    s.date,
    s.ga_session_id,
    sp.country,
    sp.device,
    sp.continent,
    sp.channel,
    ab.test,
    ab.test_group
  FROM
    `DA.ab_test` ab
  JOIN
    `DA.session` s
  ON
    ab.ga_session_id = s.ga_session_id
  JOIN
    `DA.session_params`sp
  ON
    ab.ga_session_id = sp.ga_session_id ),
  session_with_orders AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT o.ga_session_id) AS session_with_orders
  FROM
    `DA.order` o
  JOIN
    session_info
  ON
    o.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  events AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name,
    COUNT(ep.ga_session_id) AS event_cnt
  FROM
    `DA.event_params` ep
  JOIN
    session_info
  ON
    ep.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    ep.event_name ),
  session AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM
    session_info
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  account AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
  FROM
    `DA.account_session` acs
  JOIN
    session_info
  ON
    session_info.ga_session_id = acs.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group)
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session with orders' AS event_name,
  session_with_orders.session_with_orders AS value
FROM
  session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM
  events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  'session' AS event_name,
  session_cnt AS value
FROM
  session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' AS event_name,
  new_account_cnt AS value
FROM
  account
"""


In [ ]:
query_job = client.query(query)
results = query_job.result()


In [ ]:
df = results.to_dataframe()


In [ ]:
df.head()


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB


###Обчислення даних по всім тестам у загальному

In [ ]:
import pandas as pd
import statsmodels.api as sm

def analyze_ab_test(df, metrics_definitions, alpha=0.05):
    """Analyzes A/B testing results using a DataFrame.

    Args:
        df: DataFrame containing the test data.
        metrics_definitions: A list of metric names (corresponding to event_name).
        alpha: Significance level for the Z-test (default 0.05).
    """

    for test_id in df['test'].unique():
        test_df = df[df['test'] == test_id]
        print(f"\n--- Results for test {test_id} ---")

        for metric_name in metrics_definitions:
            print(f"\n--- Analyzing metric: {metric_name} ---")

            # Get metric values for each test group
            if metric_name == 'new account':
                group_values = test_df[test_df['event_name'] == metric_name].groupby('test_group')['value'].sum()
            else:
                group_values = test_df[test_df['event_name'] == metric_name].groupby('test_group')['value'].sum()

            # Get session counts for each test group
            session_values = test_df[test_df['event_name'] == 'session'].groupby('test_group')['value'].sum()

            # Check if there is enough data for both groups AND prevent ZeroDivisionError
            if len(group_values) != 2 or len(session_values) != 2 or 0 in group_values.values or 0 in session_values.values:
                print(f"Insufficient data for metric '{metric_name}' in test {test_id}. Skipping.")
                continue

            results = group_values.to_dict()
            sessions = session_values.to_dict()

            group_a = sorted(results.keys())[0]
            group_b = sorted(results.keys())[1]

            successes = [results[group_a], results[group_b]]
            nobs = [sessions[group_a], sessions[group_b]]

            z_stat, p_value = sm.stats.proportions_ztest(successes, nobs)

            print(f"Z-statistic: {z_stat}")
            print(f"P-value: {p_value}")

            significant = p_value < alpha
            print(f"Significant: {significant}")

            # Calculate Conversion Rate and Metric Change
            cr_a = successes[0] / nobs[0]
            cr_b = successes[1] / nobs[1]
            metric_change = (cr_b - cr_a) / cr_a * 100

            print(f"Conversion Rate {metric_name} {group_a}: {cr_a}")
            print(f"Conversion Rate {metric_name} {group_b}: {cr_b}")
            print(f"Metric_change_{metric_name}: {metric_change}%")

# Example usage:
metrics_definitions = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new account",
]

analyze_ab_test(df, metrics_definitions)


--- Results for test 2 ---

--- Analyzing metric: add_payment_info ---
Z-statistic: -1.240993925082954
P-value: 0.2146079930564252
Significant: False
Conversion Rate add_payment_info 1: 0.046290262061338544
Conversion Rate add_payment_info 2: 0.047946023405779796
Metric_change_add_payment_info: 3.5769107166583485%

--- Analyzing metric: add_shipping_info ---
Z-statistic: -0.7095568649433848
P-value: 0.4779789765684105
Significant: False
Conversion Rate add_shipping_info 1: 0.06872445050062208
Conversion Rate add_shipping_info 2: 0.06985908765225698
Metric_change_add_shipping_info: 1.650994869176344%

--- Analyzing metric: begin_checkout ---
Z-statistic: -0.9528980407686692
P-value: 0.3406417328714627
Significant: False
Conversion Rate begin_checkout 1: 0.08416770345794578
Conversion Rate begin_checkout 2: 0.08584109545418359
Metric_change_begin_checkout: 1.988164019556406%

--- Analyzing metric: new account ---
Z-statistic: -0.5887934728848063
P-value: 0.5559998249117384
Significant: 

###Обчислення даних по всім devices

In [ ]:
def analyze_ab_test_by_device(df, metrics_definitions, alpha=0.05):
    """Analyzes A/B testing results by device using a DataFrame.

    Args:
        df: DataFrame containing the test data.
        metrics_definitions: A list of metric names (corresponding to event_name).
        alpha: Significance level for the Z-test (default 0.05).
    """

    for device in df['device'].unique():
        device_df = df[df['device'] == device]
        print(f"\n--- Results for device: {device} ---")

        for test_id in device_df['test'].unique():
            test_df = device_df[device_df['test'] == test_id]
            print(f"\n--- Results for test {test_id} on device {device} ---")

            for metric_name in metrics_definitions:
                print(f"\n--- Analyzing metric: {metric_name} ---")

                # Get metric values for each test group
                if metric_name == 'new account':
                    group_values = test_df[test_df['event_name'] == metric_name].groupby('test_group')['value'].sum()
                else:
                    group_values = test_df[test_df['event_name'] == metric_name].groupby('test_group')['value'].sum()

                # Get session counts for each test group
                session_values = test_df[test_df['event_name'] == 'session'].groupby('test_group')['value'].sum()

                # Check for insufficient data: incorrect number of groups or zero values
                if len(group_values) != 2 or len(session_values) != 2 or 0 in group_values.values or 0 in session_values.values:
                    print(f"Insufficient data for metric '{metric_name}' in test {test_id} on device {device}. Skipping.")
                    continue

                results = group_values.to_dict()
                sessions = session_values.to_dict()

                group_a = sorted(results.keys())[0]
                group_b = sorted(results.keys())[1]

                successes = [results[group_a], results[group_b]]
                nobs = [sessions[group_a], sessions[group_b]]

                z_stat, p_value = sm.stats.proportions_ztest(successes, nobs)

                print(f"Z-statistic: {z_stat}")
                print(f"P-value: {p_value}")

                significant = p_value < alpha
                print(f"Significant: {significant}")

                # Calculate Conversion Rate and Metric Change
                cr_a = successes[0] / nobs[0]
                cr_b = successes[1] / nobs[1]
                metric_change = (cr_b - cr_a) / cr_a * 100

                print(f"Conversion Rate {metric_name} {group_a}: {cr_a}")
                print(f"Conversion Rate {metric_name} {group_b}: {cr_b}")
                print(f"Metric_change_{metric_name}: {metric_change}%")

# Example usage:
metrics_definitions = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new account",
]

analyze_ab_test_by_device(df, metrics_definitions)


--- Results for device: mobile ---

--- Results for test 2 on device mobile ---

--- Analyzing metric: add_payment_info ---
Z-statistic: 0.09956172763876983
P-value: 0.9206922797202424
Significant: False
Conversion Rate add_payment_info 1: 0.048858470300244795
Conversion Rate add_payment_info 2: 0.04864345009111156
Metric_change_add_payment_info: -0.44008788611656047%

--- Analyzing metric: add_shipping_info ---
Z-statistic: 0.8427543248281288
P-value: 0.39936585774587097
Significant: False
Conversion Rate add_shipping_info 1: 0.0707898286456512
Conversion Rate add_shipping_info 2: 0.06863737598704191
Metric_change_add_shipping_info: -3.0406241967101044%

--- Analyzing metric: begin_checkout ---
Z-statistic: 1.5654232850616396
P-value: 0.11748369217113229
Significant: False
Conversion Rate begin_checkout 1: 0.08822500874256882
Conversion Rate begin_checkout 2: 0.08382263616116623
Metric_change_begin_checkout: -4.989937257187753%

--- Analyzing metric: new account ---
Z-statistic: -1.8

###Обчислення даних по всім channels

In [ ]:
def analyze_ab_test_by_channel(df, metrics_definitions, alpha=0.05):
    """Analyzes A/B testing results by channel.

    Args:
        df: DataFrame containing the test data.
        metrics_definitions: A list of metric names (corresponding to event_name).
        alpha: Significance level for the Z-test (default 0.05).
    """

    for channel in df['channel'].unique():
        channel_df = df[df['channel'] == channel]
        print(f"\n--- Results for channel: {channel} ---")

        for test_id in channel_df['test'].unique():
            test_df = channel_df[channel_df['test'] == test_id]
            print(f"\n--- Results for test {test_id} on channel {channel} ---")

            for metric_name in metrics_definitions:
                print(f"\n--- Analyzing metric: {metric_name} ---")

                # Get metric values for each test group
                if metric_name == 'new account':
                    group_values = test_df[test_df['event_name'] == metric_name].groupby('test_group')['value'].sum()
                else:
                    group_values = test_df[test_df['event_name'] == metric_name].groupby('test_group')['value'].sum()

                # Get session counts for each test group
                session_values = test_df[test_df['event_name'] == 'session'].groupby('test_group')['value'].sum()

                # Check for insufficient data: incorrect number of groups or zero values
                if len(group_values) != 2 or len(session_values) != 2 or 0 in group_values.values or 0 in session_values.values:
                    print(f"Insufficient data for metric '{metric_name}' in test {test_id} on channel {channel}. Skipping.")
                    continue

                results = group_values.to_dict()
                sessions = session_values.to_dict()

                group_a = sorted(results.keys())[0]
                group_b = sorted(results.keys())[1]

                successes = [results[group_a], results[group_b]]
                nobs = [sessions[group_a], sessions[group_b]]

                z_stat, p_value = sm.stats.proportions_ztest(successes, nobs)

                print(f"Z-statistic: {z_stat}")
                print(f"P-value: {p_value}")

                significant = p_value < alpha
                print(f"Significant: {significant}")

                # Calculate Conversion Rate and Metric Change
                cr_a = successes[0] / nobs[0]
                cr_b = successes[1] / nobs[1]
                metric_change = (cr_b - cr_a) / cr_a * 100

                print(f"Conversion Rate {metric_name} {group_a}: {cr_a}")
                print(f"Conversion Rate {metric_name} {group_b}: {cr_b}")
                print(f"Metric_change_{metric_name}: {metric_change}%")

# Example usage:
metrics_definitions = [
    "add_payment_info",
    "add_shipping_info",
    "begin_checkout",
    "new account",
]

analyze_ab_test_by_channel(df, metrics_definitions)


--- Results for channel: Organic Search ---

--- Results for test 2 on channel Organic Search ---

--- Analyzing metric: add_payment_info ---
Z-statistic: 2.820279339590393
P-value: 0.004798186069486433
Significant: True
Conversion Rate add_payment_info 1: 0.039963357380052675
Conversion Rate add_payment_info 2: 0.034255221482671565
Metric_change_add_payment_info: -14.283424295653074%

--- Analyzing metric: add_shipping_info ---
Z-statistic: 1.9426711977024618
P-value: 0.05205589839540171
Significant: False
Conversion Rate add_shipping_info 1: 0.06469712584449788
Conversion Rate add_shipping_info 2: 0.05967408767500574
Metric_change_add_shipping_info: -7.7639278467566175%

--- Analyzing metric: begin_checkout ---
Z-statistic: 2.656343084453847
P-value: 0.007899320372393971
Significant: True
Conversion Rate begin_checkout 1: 0.07929691972976068
Conversion Rate begin_checkout 2: 0.0717810420013771
Metric_change_begin_checkout: -9.478145877541344%

--- Analyzing metric: new account ---
Z

### Етапи роботи та висновки:
Був створений Portfolio Project 1 за допомогою інструментів: SQL, Googlesheets, Python, Tableau.
Спочатку в SQL був створений запит з бази даних, у якому міститься інформація про необхідні показники. З BiqQuery я завантажила дані у Tableau та на основі їх створила дашборд. У дашборді містяться дані про AB testing.

https://public.tableau.com/app/profile/olena.halkina6622/viz/ABTesting_17365330231360/ABtest?publish=yes

Наступним етапом було розрахування показників по 4-м метрикам add_payment_info / session, add_shipping_info / session, begin_checkout / session, new_accounts / session у Python. Спочатку я підвантажила у Python базу даних з SQL. За допомогою циклів я знайшла Z-statistic, P-value, Conversion_rate варіанту А, Conversion_rate варіанту В, Metric_change. Сonversion_rate знаходяться по формулі: cr_a = successes[0] / nobs[0], cr_b = successes[1] / nobs[1],Metric_change знаходиться по формулі: metric_change = (cr_b - cr_a) / cr_a * 100  Я знайшла значення усіх цих показників для 4-х метрик у розрізі 4-х тестів та по пристроях та по каналам. Результат я перенесла у Googlesheets. https://docs.google.com/spreadsheets/d/17E8YLzDl8MwkLhoDBLh0rFCps_DF6ZRYge-OpGxgqyo/edit?usp=sharing
Наступним кроком було підключити цю таблицю до Tableau. Я створила новий дашборд Metrics, у якому розмістилися візуалізації по показникам для 4-х метрик. Показник Conversion Rate Test % я розрахувала у Tableau за допомогою формули: SUM([Numerator Count A]) / SUM([Denominator Count A])* 100. Мій загальний дашборд має 3 сторінки: AB test, Calculator, Metrics, які перемикаються за допомогою навігації. На сторінці Metrics можна відстежити показники у розрізі 4-х тестів та по пристроях для всіх 4-х тестів.

